# Simeple about token staking and reward

> 模拟简单模型的staking和reward的逻辑和计算

In [1]:
import calendar
import time

In [3]:
calendar.timegm(time.gmtime())

1675223848

In [ ]:
class SimepleStakingRewards(object):
    
    def __init__(self) -> None:
        self.stakingToken = 'stakingToken'
        self.rewardsToken = 'rewardsToken'

        self.finishAt = 0
        self.updatedAt = 0
        self.duration = 1 * 60*60*24 # seconds

        self.rewardRate = 0
        self.rewardPerTokenStored = 0

        self.userRewardPerTokenPaid = {'address': 0}
        self.rewards = {'address': 0}

        self.totalSupply = 0
        self.balanceOf = {'address': 0}
    
    def stake(self, address: str, amount: int):
        self.updateReward(address)
        self.balanceOf[address] += amount
        self.totalSupply += amount

    def updateReward(self, _account: str):
        rewardPerTokenStored = self.rewardPerToken()
        updatedAt = self.lastTimeRewardApplicable()

        if _account != "address_0":
            self.rewards[_account] = self.earned(_account)
            self.userRewardPerTokenPaid[_account] = rewardPerTokenStored
    
    def withdraw(self, to: str, amount: int):
        self.balanceOf[to] -= amount
        self.totalSupply -= amount
        return amount
    
    def earn(self, _account: str):
        return ((self.balanceOf[_account] *
                (self.rewardPerToken() - self.userRewardPerTokenPaid[_account])) / 1e18) + self.rewards[_account]

    def getReward(self, to: str):
        self.updateReward(to)
        reward = self.rewards[to]
        if reward > 0:
            self.rewards[to] = 0
            # rewardsToken.transfer(to, reward)
        
        return reward

    def rewardPerToken(self):
        if self.totalSupply == 0:
            return self.rewardPerTokenStored
        else:
            return self.rewardPerTokenStored + (self.rewardRate * (self.lastTimeRewardApplicable() - self.updatedAt) * 1e18) / self.totalSupply

    def notifyRewardAmount(self, _amount: int):
        self.updateReward("address_0")
        if self.blockTimestamp() >= self.finishAt:
            self.rewardRate = _amount / self.duration
        else:
            remainingRewards = (self.finishAt - self.blockTimestamp()) * self.rewardRate
            self.rewardRate = (_amount + remainingRewards) / self.duration
        

        assert(self.rewardRate > 0, "reward rate = 0")

        self.finishAt = self.blockTimestamp() + self.duration
        self.updatedAt = self.blockTimestamp()
    
    def lastTimeRewardApplicable(self):
        return self.finishAt if self.finishAt > self.blockTimestamp() else self.blockTimestamp()

    def blockTimestamp(self):
        """Akin to `block.timestamp`"""
        return calendar.timegm(time.gmtime())

